# Module 4 Exercises


###  Question 1.
The following Python script scrapes payroll data from the following website: http://kanview.ks.gov/

"KanView is an online solution that brings better visibility, openness, and accountability to Kansas State Government. KanView contains data related to Government spending and income, allowing Kansas taxpayers an inside look at Government financial activity."

The script scrapes the payroll data for the Fort Hays State University (https://www.fhsu.edu/) using Selenium drive along with Beautiful soup parsing library. It stores the scraped results in a Pandas DataFrame. It converts the DataFrame into a corresponding JSON object. It finally writes the JSON object into a json file in your local file system.

Modify this script to scrape the wages data for the University of Kansas (https://ku.edu/)

In [ ]:
%%bash
chmod 777 /tmp
mkdir data
apt-get update --allow-unauthenticated 
apt-get update -y --fix-missing 
pip install selenium
apt-get install chromium-chromedriver -y --fix-missing
pip install joblib
apt-get update --fix-missing

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [985 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


In [ ]:
from selenium.webdriver.common.by import By
## Headless Browser with BeautifulSoup
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import re                      # for handling regular expressions
import pandas as pd            # using pandas library
from tabulate import tabulate  # tabulate, which takes a list of lists or another tabular data type as the first argument, and outputs a nicely formatted plain-text table

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = '/usr/bin/chromium-browser'

driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver', options = options)
driver.get("http://kanview.ks.gov/PayRates/PayRates_Agency.aspx")
#"http://kanview.ks.gov/PayRates/PayRates_Agency.aspx"
#https://ku.edu/

#print('******************************',a)

#After opening the url above, Selenium clicks the specific agency link, in this case KSU (Kansas State University)
python_button = driver.find_element(By.ID, "MainContent_uxLevel1_Agencies_uxAgencyBtn_51") #Kansas State University
python_button.click() #click fhsu link


#Selenium hands the page source to Beautiful Soup
soup_level1=BeautifulSoup(driver.page_source, 'lxml')


datalist = [] #empty list
x = 0 #counter

#Beautiful Soup finds all Job Title links on the agency page and the loop begins
for link in soup_level1.find_all('a', id=re.compile("MainContent_uxLevel2_JobTitles_uxJobTitleBtn_")):
    
    #Selenium visits each Job Title page
    python_button = driver.find_element(By.ID, 'MainContent_uxLevel2_JobTitles_uxJobTitleBtn_' + str(x))
    python_button.click() #click link
    
    #Selenium hands of the source of the specific job page to Beautiful Soup
    soup_level2=BeautifulSoup(driver.page_source, 'lxml')

    #Beautiful Soup grabs the HTML table on the page
    table = soup_level2.find_all('table')[0]
    
    #Giving the HTML table to pandas to put in a dataframe object
    df = pd.read_html(str(table),header=0)
    
    #Store the dataframe in a list
    datalist.append(df[0])
    
    #Ask Selenium to click the back button
    driver.execute_script("window.history.go(-1)") 
    
    #increment the counter variable before starting the loop over
    x += 1
    
    #end loop block
    
#loop has completed

#end the Selenium browser session
driver.quit()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


In [ ]:
#combine all pandas dataframes in the list into one big dataframe
result = pd.concat([pd.DataFrame(datalist[i]) for i in range(len(datalist))],ignore_index=True)

#convert the pandas dataframe to JSON
json_records = result.to_json(orient='records')

#pretty print to CLI with tabulate
#converts to an ascii table
print(tabulate(result, headers=["Employee Name","Job Title","Overtime Pay","Total Gross Pay"],tablefmt='psql'))


+------+---------------------------------------------+--------------------------------+----------------+-------------------+
|      | Employee Name                               | Job Title                      | Overtime Pay   | Total Gross Pay   |
|------+---------------------------------------------+--------------------------------+----------------+-------------------|
|    0 | Adams-Wright, Gayla Anne                    | Academic Advisor               | $0.00          | $53,520.13        |
|    1 | Allen, Sherryl Jacqueline                   | Academic Advisor               | $0.00          | $41,186.60        |
|    2 | Archer, Elizabeth Swanson                   | Academic Advisor               | $0.00          | $45,780.00        |
|    3 | Avalos, Sandra D                            | Academic Advisor               | $0.00          | $42,614.86        |
|    4 | Bachura, Gabriel John                       | Academic Advisor               | $0.00          | $42,024.84        |


In [ ]:
#open, writem and download the file to your local file system
from google.colab import files

with open('payroll_data.json', 'w') as f:
  f.write(json_records)

files.download('payroll_data.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
result

,Employee Name,Job Title,Overtime Pay,Total Gross Pay
0,"Adams-Wright, Gayla Anne",Academic Advisor,$0.00,"$53,520.13"
1,"Allen, Sherryl Jacqueline",Academic Advisor,$0.00,"$41,186.60"
2,"Archer, Elizabeth Swanson",Academic Advisor,$0.00,"$45,780.00"
3,"Avalos, Sandra D",Academic Advisor,$0.00,"$42,614.86"
4,"Bachura, Gabriel John",Academic Advisor,$0.00,"$42,024.84"
...,...,...,...,...
4971,"Johnson, Dallas E",Visiting Professor,$0.00,"$19,440.00"
4972,"Rivera, Lysa Marta",Visiting Professor,$0.00,"$8,000.00"
4973,"Tushabe, Caroline Mercy",Visiting Professor,$0.00,"$3,900.00"
4974,"Davidson, Jeffrey L",Watershed Project Specialist,$0.00,"$61,758.84"


In [ ]:
result['Total Gross Pay']

0       $53,520.13
1       $41,186.60
2       $45,780.00
3       $42,614.86
4       $42,024.84
           ...    
4971    $19,440.00
4972     $8,000.00
4973     $3,900.00
4974    $61,758.84
4975    $36,891.17
Name: Total Gross Pay, Length: 4976, dtype: object

### Question 2.

Using .decribe() method in Pandas (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html), generate descriptive statistics for Total Gross Pay and Overtime Pay.



In [ ]:
from traitlets.traitlets import describe
# Answer:

# you may find the following stackoverflow info helpful
# https://stackoverflow.com/questions/32464280/converting-currency-with-to-numbers-in-python-pandas

result[['Total Gross Pay', 'Overtime Pay']].describe(include='all')


,Total Gross Pay,Overtime Pay
count,4976,4976
unique,4688,540
top,"$47,476.00",$0.00
freq,10,4431


### Question 3.
Using Pandas, calculate the average Total Gross Pay for Education Program Coordinator. Should be "Program/Project Coordinator" !

In [ ]:
from pandas.core.generic import Axis
from numpy import float16
# Answer: 
# Hint: You may find .loc[] function helpful to extract portions of your dataframe
result.loc[:, 'Total Gross Pay'] = result.loc[:, 'Total Gross Pay'].replace('[\$,]', '', regex=True).astype(float)
#result.head(5)
#result[result['Job Title'] == 'Program/Project Coordinator'].head(100)
print('%.2f' %result[result['Job Title'] == 'Program/Project Coordinator'].loc[:, 'Total Gross Pay'].mean())
#result.loc[:, 'Total Gross Pay'].mean()



41795.40
